# 

In [1]:
import os; from pyrus_marketbook import pyrus_marketbook;
from pyrus_marketbook import ob;

nGen=20000; #nGen = 5000; 
nMidPoints = 4000;
import numpy as np; import pandas as pd;
import pyarrow as pa;
out_t, nr, v_d_fake, bprices, sprices, \
  write_sum_q, write_sum_pq, write_avp, write_worstpi, \
  write_nlevels, write_atq, \
  v_w, verbose, time_minmax, a_Fake_OB, MidTable = \
  ob.fake_for_algo(nGen = nGen,nMidPoints=nMidPoints,AddMidPrice=True, deltaSec=.01, v_bhit=False,fast_check="PARALLEL");



 pyrus_marketbook first try to from import
------------------- pyrus_marketbook:__init__.py  running ---------- ---
-- dir(pyrus_makretbook) is [__all__,__doc__,__file__,__loader__,__name__,__package__,__spec__,asjoin_py,b2v_test,cumulate,dynarray,dynarray2,example_take,limit_hit,out_recarray,out_tmprsd,pyrus_bbo_algo,pyrus_verify_prices,raw_string_fun_example,sip_algo,sip_arrow,take_table,unsorted_asjoin_py]
--- Thank you for running init.
-- dir(pb) is [__all__,__doc__,__file__,__loader__,__name__,__package__,__spec__,asjoin_py,b2v_test,cumulate,dynarray,dynarray2,example_take,limit_hit,out_recarray,out_tmprsd,pyrus_bbo_algo,pyrus_verify_prices,raw_string_fun_example,sip_algo,sip_arrow,take_table,unsorted_asjoin_py]
-------------------------------------------------------------
Fake_OB(Vb=1,nG=20000):  -- Begin
Fake_OB(Vb=1,nG=20000):  -- Fake_OB Generated mid_of_v, now makign add points
Fake_OB(Vb=1,nG=20000): : testing v_t1lc length 30000 for data against midpoints at 6000 points.
F

In [2]:
out_t['time'] = out_t['time'].astype('datetime64[ns]')
kalgoint_basic = np.array([0], dtype=np.uint8)[0];
v_d = np.asarray([1], dtype=np.int64); verbose_book = 0;
import time; 
time_00 = time.time();
out_PMB = pyrus_marketbook.pyrus_bbo_algo(out_t, nr, v_d, 
  bprices, sprices, kalgoint_basic,
  1, 0, 0, 0, 0, 0,
  v_w, verbose_book, time_minmax.astype(np.int64) );
time_01 = time.time();
print("pd_dem calculated from " + str(len(out_t)) + " recs in " + str(time_01-time_00) + " sec.");
pd_dem = pa.Table.from_batches([out_PMB]).to_pandas();
pd_dem = pd_dem[(pd_dem['best_bid'] > 0) & (pd_dem['best_ask'] > 0)].reset_index(inplace=False, drop=True)

pd_dem calculated from 40360 recs in 0.09830570220947266 sec.


In [3]:
nTrades = 200; trade_i = np.asarray(np.unique(np.random.choice(np.arange(len(pd_dem)), nTrades, replace=False)))
ts = np.random.choice(np.asarray([0,1]),nTrades)
trades =  pd_dem.iloc[trade_i].reset_index(inplace=False, drop=True);
tprice = trades['best_bid'].to_numpy();
tprice[ts == 0] == trades['best_ask'].to_numpy()[ts == 0]
tqty = trades['b_sum_q_1'];  tqty[ts==0] == trades['s_sum_q_1'][ts==0];
trades = pd.DataFrame({'time':trades['time'],'price':tprice,'qty': (-2*ts + 1) * tqty})
trades['time'] = (trades['time']-pd_dem['time'][0]).astype(np.int64)/1000000000.0

trades

,time,price,qty
0,0.050292,9996.0,-539.0
1,0.155613,9985.0,140.0
2,0.176932,9984.0,900.0
3,0.366902,9988.0,-225.0
4,0.429516,9981.0,-390.0
...,...,...,...
195,39.138253,9981.0,-449.0
196,39.212435,9981.0,449.0
197,39.295661,9986.0,-329.0
198,39.445636,9970.0,1080.0


In [5]:
dt = str(pd_dem['time'][0])[0:10]; st_time =  str(pd_dem['time'][0])[11:26]; unit = 0;
time0 = pd_dem['time'][0];
nbbo = pd_dem[['time','best_bid','best_ask']].reset_index(inplace=False, drop=True); nbbo.columns = ['time','nbb','nbo']; nbbo['time'] = ((nbbo['time'] - nbbo['time'][0]).astype(np.int64)) / 1000000000.0
tmin = np.min(nbbo['time']); tmax = np.max(nbbo['time']);
inner_pmin = np.min(nbbo['nbb']);  inner_pmax = np.max(nbbo['nbo']);
pmin = inner_pmin - 2 * (inner_pmax - inner_pmin);  pmax = inner_pmax + 2 * (inner_pmax - inner_pmin);
height = 400; width = 800;
lwd_h = .01; lwd_w = (400/800) * lwd_h;
max_qty = np.max(out_t['cm_qty']); pow_qty = .25;
trade_mul_fac=.1; msg_mul_fac = .1;
buys = a_Fake_OB[a_Fake_OB['bs'] == b'b'][['price','qty','t0','t1']].reset_index(inplace=False, drop=True); buys['t0'] = (buys['t0'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  buys['t1'] = (buys['t1'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
buys.columns = ['price', 'qty', 'open','close']
buys

sells = a_Fake_OB[a_Fake_OB['bs'] == b's'][['price','qty','t0','t1']].reset_index(inplace=False, drop=True); sells['t0'] = (sells['t0'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  sells['t1'] = (sells['t1'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
sells.columns = ['price', 'qty', 'open','close']
sells
data_type = 'generated_data';

In [24]:
out_t
mxtime = np.max(out_t['time']);
buys = out_t[out_t['side'] == b'b'][['price','cm_qty','time']].reset_index(inplace=False,drop=True);
close = buys['time'].shift(-1,fill_value=mxtime);
close[buys['price'] != buys['price'].shift(-1,fill_value=-1)] = mxtime; buys['close'] = close;  buys.columns = ['price','qty','open','close'];  
buys['open'] = (buys['open'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  
buys['close'] = (buys['close'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
buys = buys[buys.qty > 0].reset_index(inplace=False, drop=True)

sells = out_t[out_t['side'] == b's'][['price','cm_qty','time']].reset_index(inplace=False,drop=True);
close = sells['time'].shift(-1,fill_value=mxtime);
close[sells['price'] != sells['price'].shift(-1,fill_value=-1)] = mxtime; sells['close'] = close;  sells.columns = ['price','qty','open','close'];  
sells['open'] = (sells['open'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  
sells['close'] = (sells['close'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
sells = sells[sells.qty > 0].reset_index(inplace=False, drop=True);
sells

,price,qty,open,close
0,9969.0,403.0,1.550386,1.555701
1,9970.0,294.0,1.551404,1.555880
2,9970.0,644.0,1.678956,1.679315
3,9971.0,68.0,1.171931,1.177928
4,9972.0,491.0,39.433239,39.437356
...,...,...,...,...
17097,10258.0,276.0,14.789861,17.050698
17098,10258.0,944.0,17.050698,40.000000
17099,10258.0,276.0,40.000000,40.000000
17100,10281.0,632.0,13.950663,40.000000


In [26]:
test_data = { "dt": dt, 'st_time':st_time, 'unit':unit, 'tmin':tmin, 'tmax':tmax, 'pmin':pmin, 'pmax':pmax,'height':height, 'width':width,
                'lwd_h':lwd_h, 'lwd_w':lwd_w, 'max_qty':max_qty, 'pow_qty':pow_qty,'trade_mul_fac':trade_mul_fac, 'msg_mul_fac':msg_mul_fac,
                'nbbo':nbbo.to_dict(orient='list'),'buys':buys.to_dict(orient='list'),'sells':sells.to_dict(orient='list'),'trades':trades.to_dict(orient='list'),'data_type':'generated_data'};
import json;
aD = json.dumps(test_data)

In [86]:
import pathlib
import anywidget
import traitlets
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }</style>"))


from IPython.display import HTML, display
html_code = "<h1>Hello, world!</h1><p>This is a paragraph.</p>"
install_div_space = \
"""<div id="gl_div" name="gl_div"> <div>"""
display(HTML(install_div_space))

class obwidget(anywidget.AnyWidget):
    _esm = "./outwidget/static/widget_index.js"
    _css = "./outwidget/static/widget_index.css"
    data = traitlets.Dict(test_data).tag(sync=True)
    value = traitlets.Int(0).tag(sync=True)

obwidget()

obwidget(data={'dt': '2024-10-03', 'st_time': '10:30:32', 'unit': 0, 'tmin': 0.0, 'tmax': 39.992168448, 'pmin'…

In [8]:
import pathlib
import anywidget
import traitlets
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }</style>"))


from IPython.display import HTML, display
html_code = "<h1>Hello, world!</h1><p>This is a paragraph.</p>"
install_div_space = \
"""<div id="gl_div" name="gl_div"> <div>"""
display(HTML(install_div_space))

class obwidget(anywidget.AnyWidget):
    _esm = "./outwidget/static/widget_index.js"
    _css = "./outwidget/static/widget_index.css"
    value = traitlets.Int(0).tag(sync=True)

obwidget()

obwidget()